In [1]:
url = "http://www.nasdaq.com/earnings/earnings-calendar.aspx?date=2017-Nov-20"
frame = '<iframe src=' + url + 'width=700 height=500></iframe>'

In [2]:
from bs4 import BeautifulSoup
import urllib
r = urllib.urlopen(url).read()
soup = BeautifulSoup(r)
print type(soup)

<class 'bs4.BeautifulSoup'>


C:\ProgramData\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\ProgramData\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [3]:
print soup.prettify()[0:1000]

<!DOCTYPE html>
<html class="inner no-js" lang="en-us" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-K2BQVP7');
  </script>
  <!-- End Google Tag Manager -->
  <script>
   (function(){
var is_chrome;
if(typeof navigator.vendor!="undefined")
	is_chrome = ((navigator.userAgent.toLowerCase().indexOf('chrome') > -1) &&(navigator.vendor.toLowerCase().indexOf("google") > -1));
else
	is_chrome = false;
if(is_chrome)
{
//instart
//Copyright Intart Logic Thu Nov 16 2017 - version: 8.3.21
!function(t){if(function(){var n=!0;try{var i="object"==typeof t.IXC_320_8513696329873064


In [4]:
companiesTable = soup.find(id='ECCompaniesTable')
print(type(companiesTable))

<class 'bs4.element.Tag'>


In [5]:
from enum import Enum
class Hours(Enum):
    DAY = 1
    NIGHT = 2
    UNKNOWN = 3

In [6]:
rows = companiesTable.findChildren(['th', 'tr'])

In [7]:
def parse_time_col(time_child):
    if len(time_child.select('a')) == 0:
        return Hours.UNKNOWN
    
    title = time_child.select('a')[0]['title']
    if title == 'After Hours Quotes':
        return Hours.NIGHT
    if title == 'Pre-market Quotes':
        return Hours.DAY
    return Hours.UNKNOWN

In [8]:
from re import sub
from decimal import Decimal

def parse_decimal(money):
    if money == '$n/a':
        return None
    return Decimal(money[1:])

In [9]:
import pandas as pd
from datetime import datetime

def parse_table():
    column_titles = ['Time', 'Symbol', 'Reported Date', 'Quarter', 'ConsensusEPS', '# of Ests', 'Unknown', 'LastYearEPS', 'PercentSurprise']
    df = pd.DataFrame(columns=column_titles)

    row_marker = 0
    for row in rows:
        children = row.findChildren('td')
        if len(children) == 0:
            continue
            
        row_data = [column.get_text().strip() for column in children]
        row_data[0] = parse_time_col(children[0])
        row_data[2] = datetime.strptime(row_data[2], '%m/%d/%Y')
        row_data[4] = parse_decimal(row_data[4])
        row_data[5] = int(row_data[5])
        if row_data[6] == 'n/a':
            row_data[6] = None
        else:
            row_data[6] = datetime.strptime(row_data[6], '%m/%d/%Y')
        row_data[7] = parse_decimal(row_data[7])
            
        df.loc[row_marker] = row_data
        row_marker += 1

    return df

In [10]:
df = parse_table()
df.tail()

,Time,Symbol,Reported Date,Quarter,ConsensusEPS,# of Ests,Unknown,LastYearEPS,PercentSurprise
16,Hours.NIGHT,"Tarena International, Inc. (TEDU) Market Cap: ...",2017-11-20,Sep 2017,None,0,2016-11-21,0.31,Met
17,Hours.NIGHT,"Urban Outfitters, Inc. (URBN) Market Cap: $2.91B",2017-11-20,Oct 2017,0.33,14,2016-11-22,0.4,Met
18,Hours.DAY,UTStarcom Holdings Corp (UTSI) Market Cap: $11...,2017-11-20,Sep 2017,None,0,2016-11-08,-0.05,Met
19,Hours.NIGHT,Vipshop Holdings Limited (VIPS) Market Cap: $5...,2017-11-20,Sep 2017,0.1,1,2016-11-21,0.12,Met
20,Hours.NIGHT,ZTO Express (Cayman) Inc. (ZTO) Market Cap: $1...,2017-11-20,Sep 2017,0.15,1,2016-11-28,0.12,Met


In [11]:
filtered_df = df[(df['Time'] == Hours.NIGHT) & (df['ConsensusEPS'] > df['LastYearEPS'])]
# filtered_df = df[(df['Time'] == Hours.DAY) & (df['ConsensusEPS'] > df['LastYearEPS'])]
filtered_df

,Time,Symbol,Reported Date,Quarter,ConsensusEPS,# of Ests,Unknown,LastYearEPS,PercentSurprise
0,Hours.NIGHT,"Agilent Technologies, Inc. (A) Market Cap: $22...",2017-11-20,Oct 2017,0.63,9,2016-11-15,0.59,Met
1,Hours.NIGHT,"Amtech Systems, Inc. (ASYS) Market Cap: $194.55M",2017-11-20,Sep 2017,0.05,2,2016-11-17,-0.02,Met
2,Hours.NIGHT,"Beacon Roofing Supply, Inc. (BECN) Market Cap:...",2017-11-20,Sep 2017,0.94,12,2016-11-21,0.88,Met
4,Hours.NIGHT,Cubic Corporation (CUB) Market Cap: $1.44B,2017-11-20,Sep 2017,0.14,2,2016-11-21,-0.29,Met
5,Hours.NIGHT,"Enanta Pharmaceuticals, Inc. (ENTA) Market Cap...",2017-11-20,Sep 2017,1.94,2,2016-11-21,-0.09,Met
13,Hours.NIGHT,"Palo Alto Networks, Inc. (PANW) Market Cap: $1...",2017-11-20,Oct 2017,-0.24,4,2016-11-21,-0.58,Met
14,Hours.NIGHT,"Raven Industries, Inc. (RAVN) Market Cap: $1.21B",2017-11-20,Oct 2017,0.21,3,2017-02-15,0.16,Met
20,Hours.NIGHT,ZTO Express (Cayman) Inc. (ZTO) Market Cap: $1...,2017-11-20,Sep 2017,0.15,1,2016-11-28,0.12,Met


In [12]:
symbols = [s[s.find("(")+1:s.find(")")] for s in filtered_df['Symbol']]
print len(symbols)

8


In [13]:
symbols

[u'A', u'ASYS', u'BECN', u'CUB', u'ENTA', u'PANW', u'RAVN', u'Cayman']

In [14]:
filtered_df.describe()

,Time,Symbol,Reported Date,Quarter,ConsensusEPS,# of Ests,Unknown,LastYearEPS,PercentSurprise
count,8,8,8,8,8,8.0,8,8,8
unique,1,8,1,2,8,6.0,5,8,1
top,Hours.NIGHT,ZTO Express (Cayman) Inc. (ZTO) Market Cap: $1...,2017-11-20 00:00:00,Sep 2017,0.63,2.0,2016-11-21 00:00:00,0.12,Met
freq,8,1,8,5,1,3.0,4,1,8
first,NaN,NaN,2017-11-20 00:00:00,NaN,NaN,NaN,2016-11-15 00:00:00,NaN,NaN
last,NaN,NaN,2017-11-20 00:00:00,NaN,NaN,NaN,2017-02-15 00:00:00,NaN,NaN
